In [1]:
# ice-cream example

states = ('HOT', 'COLD')
symbols = ('1', '2', '3')

start_prob = {
    'HOT': 0.8,
    'COLD': 0.2
}

trans_prob = {
    'HOT': { 'HOT': 0.6, 'COLD': 0.4 },
    'COLD': { 'HOT': 0.5, 'COLD': 0.5 },
}

emit_prob = {
    'HOT': { '1': 0.2, '2': 0.4, '3': 0.4 },
    'COLD': { '1': 0.5, '2': 0.4, '3': 0.1 },
}

sequence = ['3', '1', '3']

In [2]:
# def forward()
# def decode()
# def backward()
# def learn()

### Likelihood Computation: The Forward Algorithm

- the likelihood of the observation sequence is:
$$P(O|Q) = \prod_{i=1}^{T} P(o_i|q_i)$$

- joint probability:
$$P(O,Q) = P(O|Q) \times P(Q) = \prod_{i=1}^{T} P(o_i|q_i) \times \prod_{i=1}^{T} P(p_i|q_{i-1})$$

- total probability of the observations:
$$P(O) = \sum_Q P(O,Q) = \sum_Q P(O|Q)P(Q)$$

For an HMM with $N$ hidden states and an observation sequence of $T$ observations, there are $N^T$ possible hidden sequences.

#### forward algorithm - $O(N^2T)$
- Each cell of the forward algorithm trellis $\alpha_t(j)$ represents the probability of being in state $j$ after seeing the first $t$ observations, given the automaton $\lambda$. The value of each cell $\alpha_t(j)$ is computed by summing over the probabilities of every path.
$$\alpha_t(j) = P(o_1, o_2, \dots o_t, q_t = j | \lambda)$$
- For a given state $q_j$ at time $t$, the value $\alpha_t(j)$ is computed as:
$$\alpha_t(j) = \sum_{i=1}^{N} \alpha_{t-1}(i) a_{ij} b_j(o_t)$$
- The three factors:
 - $\alpha_{t-1}(i) ~~~~~ : ~$ the **previous forward path probability**
 - $a_{ij} ~~~~~~~~~~~~ : ~$the **transition probability**
 - $b_j(o_t) ~~~~~~~ : ~$ the **state observation likelihood**

1. Initialization:
$$\alpha_1(j) = \pi_j b_j(o_1) \quad 1\le j \le N$$
2. Recursion:
$$\alpha_t(j) = \sum_{i=1}^{N} \alpha_{t-1}(i) a_{ij} b_j(o_t); \quad 1 \le j \le N, 1 < t \le T$$
3. Termination:
$$P(O|\lambda) = \sum_{i=1}^N \alpha_T(i)$$

In [3]:
def forward(states, symbols, start_prob, trans_prob, emit_prob, sequence):
    length = len(sequence)
    
    # t == 1, alpha_1(j) = pi_j * b_j(o_1)
    alpha = [{}]
    for state in states:
        alpha[0][state] = start_prob[state] * emit_prob[state][sequence[0]]
    
    # t >= 2, alpha_t(j) = sum_i_N   alpha_t-1(i) * a_ij * b_j(o_t)
    for t in range(1, length):
        alpha.append({})
        for state_j in states:
            prob = 0
            for state_i in states:
                prob += alpha[t-1][state_i] * trans_prob[state_i][state_j] * emit_prob[state_j][sequence[t]]
            alpha[t][state_j] = prob
    
    return alpha

In [4]:
forward(states, symbols, start_prob, trans_prob, emit_prob, sequence)

[{'COLD': 0.020000000000000004, 'HOT': 0.32000000000000006},
 {'COLD': 0.06900000000000002, 'HOT': 0.04040000000000001},
 {'COLD': 0.005066000000000002, 'HOT': 0.02349600000000001}]

### Decoding: The Viterbi Algorithm
Given as input an HMM $\lambda = (A, B)$ and a sequence of observations $O = o_1, o_2, \dots, o_T$, find the most probable sequence of states $Q = q_1 q_2 q_3 \dots q_T$.
$$v_t(j) = \text{max}_{q_1, \dots, q_{t-1}}P(q_1 \dots q_{t-1}, o_1, o_2, \dots o_t, q_t = j | \lambda)$$
- For a given state $q_j$ at time $t$, the value $v_t(j)$ is computed as:
$$\alpha_t(j) = \text{max}_{i=1}^{N} v_{t-1}(i) a_{ij} b_j(o_t)$$
- The three factors:
 - $v_{t-1}(i) ~~~~~~ : ~$ the **previous Viterbi path probability**
 - $a_{ij} ~~~~~~~~~~~~ : ~$the **transition probability**
 - $b_j(o_t) ~~~~~~~ : ~$ the **state observation likelihood**

1. Initialization:
$$v_1(j) = \pi_j b_j(o_1) \quad 1\le j \le N$$
$$bt_1(j) = 0 \quad 1\le j \le N$$
2. Recursion:
$$v_t(j) = \text{max}_{i=1}^{N} v_{t-1}(i) a_{ij} b_j(o_t); \quad 1 \le j \le N, 1 < t \le T$$
$$bt_t(j) = \text{argmax}_{i=1}^{N} v_{t-1}(i) a_{ij} b_j(o_t); \quad 1 \le j \le N, 1 < t \le T$$
3. Termination:
$$\text{The best score:}\quad P* = \text{max}_{i=1}^N v_T(i)$$
$$\text{The start of backtrace::}\quad q_T* = \text{argmax}_{i=1}^N v_T(i)$$

In [5]:
def decode(states, symbols, start_prob, trans_prob, emit_prob, sequence):
    """Viterbi decoding"""
    length = len(sequence)
    
    # t == 1, v_1(j) = pi_j * b_j(o_1)
    # t == 1, bt_1(j) = 0
    V = {}
    for state in states:
        V[state] = start_prob[state] * emit_prob[state][sequence[0]]

    # t >= 2, v_t(j) = max_i_N   v_t-1(i) * a_ij * b_j(o_t)
    # t >= 2, bt_t(j) = argmax_i_N   v_t-1(i) * a_ij * b_j(o_t)
    paths = []
    for t in range(1, length):
        V_tmp = {}
        prev_state = {}
        for state_j in states:
            max_prob = 0
            max_state = None
            for state_i in states:
                prob = V[state_i] * trans_prob[state_i][state_j]
                if prob > max_prob:
                    max_prob = prob
                    max_state = state_i
            V_tmp[state_j] = max_prob * emit_prob[state_j][sequence[t]]
            prev_state[state_j] = max_state
        V = V_tmp
        paths.append(prev_state)
    
    # the most probable state and its backtrack
    # find the last state ("raniy", because prob of 'raniy' is bigger than 'sunny' in the last V)
    max_prob = 0
    max_state = None
    for state in states:
        if V[state] > max_prob:
            max_prob = V[state]
            max_state = state
    
    result = [max_state]
    # follow the backtrack till the first observation (T-1 ~ 1)
    for t in range(length -1, 0, -1):
        max_state = paths[t-1][max_state]
        result.insert(0, max_state)
    
    return result, V, paths

In [6]:
decode(states, symbols, start_prob, trans_prob, emit_prob, sequence)
# paths means {to: from}

(['HOT', 'COLD', 'HOT'],
 {'COLD': 0.003200000000000001, 'HOT': 0.012800000000000004},
 [{'COLD': 'HOT', 'HOT': 'HOT'}, {'COLD': 'COLD', 'HOT': 'COLD'}])

In [7]:
forward(states, symbols, start_prob, trans_prob, emit_prob, sequence)

[{'COLD': 0.020000000000000004, 'HOT': 0.32000000000000006},
 {'COLD': 0.06900000000000002, 'HOT': 0.04040000000000001},
 {'COLD': 0.005066000000000002, 'HOT': 0.02349600000000001}]

### HMM Training: The Forward-Backward Algorithm
- The backward probability $\beta$ is the probability of seeing the observations from time $t+1$ to the end, given that we are in state $i$ at time $t$ (and given the automaton $\lambda$):
$$\beta_t(i) = P(o_{t+1}, o_{t+2}, \dots o_T | q_t = i, \lambda)$$


1. Initialization:
$$\beta_T(i) = 1 \quad 1\le j \le N$$
2. Recursion:
$$\beta_t(i) = \sum_{j=1}^{N} a_{ij} b_j(o_{t+1}) \beta_{t+1}(j); \quad 1 \le i \le N, 1 \le t < T$$
3. Termination:
$$P(O|\lambda) = \sum_{j=1}^N \pi_j b_j(o_1) \beta_1(j)$$

In [8]:
def backward(states, symbols, start_prob, trans_prob, emit_prob, sequence):
    length = len(sequence)
    
    # t == T, beta_T(i) = 1
    beta = [{}]
    for state in states:
        beta[0][state] = 1
    
    # t <= T-1, beta_t(i) = sum_j_N   a_ij * b_j(o_t+1) * beta_t+1(j)
    for t in range(length-1, 0, -1):
        beta.insert(0, {})
        for state_i in states:
            prob = 0
            for state_j in states:
                prob += emit_prob[state_j][sequence[t]] * beta[1][state_j] * trans_prob[state_i][state_j]
            beta[0][state_i] = prob

    return beta

In [9]:
backward(states, symbols, start_prob, trans_prob, emit_prob, sequence)

[{'COLD': 0.0905, 'HOT': 0.08360000000000001},
 {'COLD': 0.25, 'HOT': 0.28},
 {'COLD': 1, 'HOT': 1}]

### Estimate the tarnsition probability; $\hat{a}_{ij}$
$$\hat{\alpha}_{ij} = \frac{\text{expected number of transitions from state } i \text{ to state } j}{\text{expected number of transitions from state }i}$$

- The probability of being in state $i$ at itme $t$ and state $j$ at time $t+1$, given the observations sequence and of course the model:
$$\xi_t(i, j) = P(q_t = i, q_{t+1} = j | O, \lambda)$$


- To compute $\xi_t$, we first compute a probability which is similar to $\xi_t$, but differs in including the probability of the observation; note the different conditioning of $O$:
$$\text{non-quite-}\xi_t(i, j) = P(q_t = i, q_{t+1} = j , O|\lambda)$$


- The four different probabilities are multiplied together to produce *not-quite-$\xi_t$* as follows:
$$\text{non-quite-}\xi_t(i, j) = \alpha_t(i) a_{ij} b_j(o_{t+1}) \beta_{t+1}(j)$$


- The probability of the observation given the model is simply the forward probability of the whole utterance (or the backward)
$$P(O|\lambda) = \sum_{j=1}^N \alpha_t(j) \beta_t(j)$$
- So, the final equation for $\xi_t$ is
$$\xi_t(i, j) = \frac{\alpha_t(i) a_{ij} b_j(o_{t+1}) \beta_{t+1}(j)}{\sum_{j=1}^N \alpha_t(j) \beta_t(j)}$$

$$\hat{\alpha}_{ij} = \frac{\sum_{t=1}^{T-1} \xi_t(i, j)}{\sum_{t=1}^{T-1}\sum_{k=1}^{N} \xi_t(i, k)}$$

### Estimate the observation probability;  $\hat{b}_j(v_k)$
$$\hat{b}_j(v_k) = \frac{\text{expected number of times in state } j \text{ and observing symbol } v_k}{\text{expected number of times in state }j}$$

- The probability of being in state $j$ at time $t$, which we will call $\gamma_t(j)$:
$$\gamma_t(j) = P(q_t = j | O, \lambda)$$


- Once again, we will compute this by including the observation sequence in the probability:
$$\gamma_t(j) = \frac{P(q_t = j, O | \lambda)}{P(O|\lambda)}$$


- Note that $\gamma$ is really a degenerate case of $\xi$ (state $i$ collapsed with state $j$):
$$\gamma_t(j) = \frac{\alpha_t(j) \beta_t(j)}{P(O|\lambda)}$$

- The result is the percentage of the times that we were in state $j$ and saw symbol $v_k$:
$$\hat{b}_j(v_k) = \frac{\sum_{t=1 s.t. O_t=v_k}^{T} \gamma_t(j)}{\sum_{t=1}^{T} \gamma_t(j)}$$

#### Like other cases of the EM (expectation-maximization) algorithm, the forward-backward algorithm has two steps:
- the **expectation** step, or **E-step**
 - we compute the expected state occupancy count $\gamma$ and the expected state trasition count $\xi$ from the earlier $A$ and $B$ probabilities.
- the **maximazation** step, or **M-step**
 - we use $\gamma$ and $\xi$ to recompute new $A$ and $B$ probabilities.

In [10]:
def learn(states, symbols, start_prob, trans_prob, emit_prob, sequence):
    length = len(sequence)
    alpha = forward(states, symbols, start_prob, trans_prob, emit_prob, sequence)
    beta = backward(states, symbols, start_prob, trans_prob, emit_prob, sequence)
    
    # E-step: gamma_t(j) = (alpha_t(j) * beta_t(j)) / alpha_T(q_F)
    gamma = []
    for t in range(length):
        prob_sum = 0
        gamma.append({})
        for state in states:
            prob = alpha[t][state] * beta[t][state]
            prob_sum += prob
            gamma[t][state] = prob
        # dividing
        for state in states:
            gamma[t][state] /= prob_sum
            
    # E-step: xi_t(i, j) = (alpha_t(i) * a_ij * b_j(o_t+1) * beta_t+1(j)) / alpha_T(q_F)
    xi = []
    for t in range(length-1):
        prob_sum = 0
        xi.append({})
        for state_i in states:
            xi[t][state_i] = {}
            for state_j in states:
                prob = alpha[t][state_i] * trans_prob[state_i][state_j] * \
                    emit_prob[state_j][sequence[t+1]] * beta[t+1][state_j]
                xi[t][state_i][state_j] = prob
                prob_sum += prob       
        # dividing
        for state_i in states:
            for state_j in states:
                xi[t][state_i][state_j] /= prob_sum

    return gamma, xi

In [11]:
learn(states, symbols, start_prob, trans_prob, emit_prob, sequence)

([{'COLD': 0.06337091240109236, 'HOT': 0.9366290875989077},
  {'COLD': 0.6039493032700791, 'HOT': 0.3960506967299209},
  {'COLD': 0.1773685316154331, 'HOT': 0.8226314683845669}],
 [{'COLD': {'COLD': 0.04376444226594776, 'HOT': 0.019606470135144598},
   'HOT': {'COLD': 0.5601848610041312, 'HOT': 0.3764442265947763}},
  {'COLD': {'COLD': 0.12078986065401583, 'HOT': 0.4831594426160633},
   'HOT': {'COLD': 0.05657867096141726, 'HOT': 0.3394720257685036}}])

In [12]:
gamma = [{'COLD': 0.06337091240109236, 'HOT': 0.9366290875989077},
         
         {'COLD': 0.6039493032700791, 'HOT': 0.3960506967299209},
      
         {'COLD': 0.1773685316154331, 'HOT': 0.8226314683845669}]

xi = [{'COLD': {'COLD': 0.04376444226594776, 'HOT': 0.019606470135144598},
       'HOT': {'COLD': 0.5601848610041312, 'HOT': 0.3764442265947763}},
      
      {'COLD': {'COLD': 0.12078986065401583, 'HOT': 0.4831594426160633},
       'HOT': {'COLD': 0.05657867096141726, 'HOT': 0.3394720257685036}}]

In [13]:
# time 1 -> to cold from cold + hot = gamma cold time 1
.04376444226594776 + .019606470135144598

0.06337091240109236

In [14]:
# time 2 -> to cold from cold + hot = gamma cold time 2
.12078986065401583 + .4831594426160633

0.6039493032700791

In [15]:
# time 1 + 2 -> to cold / all prob from cold
(.04376444226594776 + .12078986065401583) / \
(.04376444226594776 +.019606470135144598 +.12078986065401583 +.4831594426160633)

0.2465897166841553

#### Derivation: $\sum_{t=1}^{T-1} \gamma_t(j) = \sum_{t=1}^{T-1}\sum_{k=1}^{N} \xi_t(i, k)$

 
1. $$\sum_{t=1}^{T-1} \gamma_t(j) = \sum_{t=1}^{T-1} \alpha_t(j) \beta_t(j)$$

2. 
$$
\begin{align}
\sum_{t=1}^{T-1}\sum_{k=1}^{N} \xi_t(i, k) &= \sum_{t=1}^{T-1}\sum_{k=1}^{N} \alpha_t(i) a_{ij} b_j(o_{t+1}) \\
&= \sum_{t=1}^{T-1} \alpha_t(i) \sum_{k=1}^{N} a_{ij} b_j(o_{t+1}) \\
&= \sum_{t=1}^{T-1} \alpha_t(j) \beta_t(j) \\
\end{align}
$$

In [16]:
# M-step: re-estimate transition prob (A)
# gamma_sum by state through time (same)

for state in states:
    gamma_sum = 0
    for index in range(2):
        gamma_sum += gamma[index][state]
#     print(gamma_sum, "({})".format(state))

    denominator = gamma_sum
    for state_to in states:
        xi_sum = 0
        for index in range(2):
            xi_sum += xi[index][state][state_to]
#         print("xi_sum: ", xi_sum, "denominater: ", denominator)
        trans_prob[state][state_to] = xi_sum / denominator

trans_prob

{'COLD': {'COLD': 0.2465897166841553, 'HOT': 0.7534102833158446},
 'HOT': {'COLD': 0.4627994955863806, 'HOT': 0.537200504413619}}

In [17]:
# M-step: re-estimate transition prob (A)
# xi_sum by state through time (same)

for state in states:
    denominator = 0
    for state_to in states:
        for index in range(2):
            denominator += xi[index][state][state_to]
    print(denominator, "({})".format(state))
            
    for state_to in states:
        xi_sum = 0
        for index in range(2):
            xi_sum += xi[index][state][state_to]
        print("xi_sum: ", xi_sum, "denominater: ", denominator)
        trans_prob[state][state_to] = xi_sum / denominator
    
trans_prob

1.3326797843288285 (HOT)
xi_sum:  0.7159162523632798 denominater:  1.3326797843288285
xi_sum:  0.6167635319655485 denominater:  1.3326797843288285
0.6673202156711715 (COLD)
xi_sum:  0.5027659127512079 denominater:  0.6673202156711715
xi_sum:  0.1645543029199636 denominater:  0.6673202156711715


{'COLD': {'COLD': 0.2465897166841553, 'HOT': 0.7534102833158446},
 'HOT': {'COLD': 0.4627994955863807, 'HOT': 0.5372005044136191}}

In [18]:
gamma = [{'COLD': 0.06337091240109236, 'HOT': 0.9366290875989077},
         
         {'COLD': 0.6039493032700791, 'HOT': 0.3960506967299209},
      
         {'COLD': 0.1773685316154331, 'HOT': 0.8226314683845669}]

xi = [{'COLD': {'COLD': 0.04376444226594776, 'HOT': 0.019606470135144598},
       'HOT': {'COLD': 0.5601848610041312, 'HOT': 0.3764442265947763}},
      
      {'COLD': {'COLD': 0.12078986065401583, 'HOT': 0.4831594426160633},
       'HOT': {'COLD': 0.05657867096141726, 'HOT': 0.3394720257685036}}]

In [19]:
# M-step: re-estimate emission prob (B)

for state in states:
    gamma_sum = 0
    for index in range(2):
        gamma_sum += gamma[index][state]
#     print(gamma_sum, "({})".format(state))

    denominator = gamma_sum
    for state_to in states:
        xi_sum = 0
        for index in range(2):
            xi_sum += xi[index][state][state_to]
#         print("xi_sum: ", xi_sum, "denominater: ", denominator)
        trans_prob[state][state_to] = xi_sum / denominator

    print("({})".format(state))
    gamma_sum += gamma[2][state]
    emit_gamma_sum = {}
    for symbol in symbols:
        emit_gamma_sum[symbol] = 0
    
    for index in range(3):
        print(gamma[index][state], "({})".format(sequence[index]))
        emit_gamma_sum[sequence[index]] += gamma[index][state]
    
    denominator = gamma_sum
    print(emit_gamma_sum, "({})".format(state), denominator)

    for symbol in symbols:
        emit_prob[state][symbol] = emit_gamma_sum[symbol] / denominator
        
emit_prob

(HOT)
0.9366290875989077 (3)
0.3960506967299209 (1)
0.8226314683845669 (3)
{'2': 0, '3': 1.7592605559834746, '1': 0.3960506967299209} (HOT) 2.1553112527133957
(COLD)
0.06337091240109236 (3)
0.6039493032700791 (1)
0.1773685316154331 (3)
{'2': 0, '3': 0.24073944401652544, '1': 0.6039493032700791} (COLD) 0.8446887472866046


{'COLD': {'1': 0.7149962695846804, '2': 0.0, '3': 0.28500373041531957},
 'HOT': {'1': 0.18375568551007146, '2': 0.0, '3': 0.8162443144899284}}

In [20]:
.9366290875989077 + .3960506967299209 + .8226314683845669

2.1553112527133957